In [1]:
# import the necessary packages
from scipy.spatial import distance as dist
from imutils import contours, perspective, grab_contours
import numpy as np
import cv2
import ipywidgets as widgets
import io
from PIL import Image
import base64

In [2]:
def measure_reference_object(ref, ref_length, dim = 'width'):
    
    # compute the rotated bounding box of the reference contour
    box = cv2.minAreaRect(ref)
    box = cv2.boxPoints(box)
    box = np.array(box, dtype="int")
    
    # order the points so that they appear in top-left, top-right,
    # bottom-right, and bottom-left order
    box = perspective.order_points(box)

    # unpack the ordered bounding box, then compute heigth and width
    (tl, tr, br, bl) = box
    height = dist.euclidean(tl, bl)
    width = dist.euclidean(tl, tr)


    # compute pixels per metric
    if dim == 'width':
        return width / ref_length
    elif dim == 'height':
        return height / ref_length
    else:
        print('Invalid dimension flag. Default to width')
        return width / ref_length

In [3]:
def compute_contours(image):
    
    # convert image to grayscale, and blur it slightly
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    
    # perform edge detection, then perform a dilation and erosion to
    # close gaps in between object edges
    edged = cv2.Canny(gray, 50, 100)
    edged = cv2.dilate(edged, None, iterations=1)
    edged = cv2.erode(edged, None, iterations=1)

    # find contours in the edge map
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = grab_contours(cnts)
    
    # sort contours
    (cnts, _) = contours.sort_contours(cnts, method="top-to-bottom")

    return cnts

In [62]:
def on_run(data):
    [uploader, width, minimum_inclusion_size, visualisation] = data

    images = {}
    for name, file_info in uploader.value.items():
        #img = Image.open(io.BytesIO(file_info['content'])).convert('RGB')
        decoded = cv2.imdecode(np.frombuffer(file_info['content'], np.uint8), -1)
        #open_cv_image = np.array(img) 
        #open_cv_image = open_cv_image[:, :, ::-1].copy() # Convert RGB to BGR 
        #decoded = decoded[:, :, ::-1].copy() # Convert RGB to BGR 
        images[name] = decoded

    image = decoded

    # process the image and find contours
    cnts = compute_contours(image)

    # calculate the 'pixels per metric' calibration variable
    pixelsPerMetric = measure_reference_object(cnts[0], width)

    # drop the reference object
    #cnts = cnts[1:] - at the moment the reference object is recognised as 2 object so the first 2 are dropped
    cnts = cnts[2:]

    name = name
    results = name

    # loop over the contours individually
    for c in cnts:

        # compute convex hull of the contour
        c = cv2.convexHull(c)
        area = cv2.contourArea(c) / np.power(pixelsPerMetric, 2)

        # if the contour is not sufficiently large, ignore it
        if area <  minimum_inclusion_size:
            continue

        # skip visualisation and store results
        if visualisation == False:
            results = results +',' + str(area) + '\n' 
            continue

        # draw the contours
        cv2.drawContours(image, c, -1, (100, 255, 255), 4)

        # add text on the image
        bottommost = tuple(c[c[:,:,1].argmax()][0])
        cv2.putText(image, "{:.2f}microns2".format(area), bottommost, cv2.FONT_HERSHEY_SIMPLEX,
                    0.65, (255, 255, 255), 2)

        # save results in a csv file
        results = results +',' + str(area) + '\n' 
    
    
    #image = image[:, :, ::-1].copy() # Convert RGB to BGR 
    is_success, im_buf_arr = cv2.imencode(".png", image)
    byte_im = im_buf_arr.tobytes()    
    
    # show the output image
    output_image.value = byte_im
    
    # create a link with results
    link_results(results)

In [63]:
def link_results(results):
   #FILE
   filename = 'chlamydia_inclusion_size.csv'
   b64 = base64.b64encode(results.encode())
   payload = b64.decode()

   #BUTTONS
   html_buttons = '''<html>

   <head>
   <meta name="viewport" content="width=device-width, initial-scale=1">
   </head>
   <body>
   <a download="{filename}" href="data:text/csv;base64,{payload}" download>
   <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download File</button>
   </a>
   </body>
   </html>
   '''

   html_button = html_buttons.format(payload=payload,filename=filename)
   display(widgets.HTML(html_button))

In [64]:
out = widgets.Output()

uploader = widgets.FileUpload(accept='.jpg, .png', multiple=False)

output_image = widgets.Image(
    value=b'',
    width = 500,
    height = 500
)

width = widgets.FloatText(
    value=10.0,
    description='Width:',
    disabled=False
)

minimum_inclusion_size=  widgets.FloatText(
    value=20.0,
    description='Minimum inclusion size:',
    disabled=False
)

visualisation = widgets.Checkbox(
    value=True,
    description='Visualisation',
    disabled=False,
    indent=False
)

button = widgets.Button(
    description='Measure chlamydia!',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

@out.capture()
def on_button_clicked(b):
    out.clear_output()
    on_run([uploader,
            width.value,
            minimum_inclusion_size.value,
            visualisation.value]
          )

button.on_click(on_button_clicked)

display(widgets.VBox([uploader, width, minimum_inclusion_size, visualisation, button, output_image, out]))